In [17]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import spectrogram, stft, istft, check_NOLA

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
from collections import OrderedDict
import os
import neptune
from neptune.utils import stringify_unsupported

plt.style.use('ggplot')

# PARAMETERS - GENERAL

In [2]:
arraySavePath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/trainTestRTheta.npz'
modelPath = '/blue/gkalamangalam/jmark.ettinger/predictScalp/pytorchModels/model1.pth'

neptuneProject = 'jettinger35/predictScalp'
api_token = os.environ.get('NEPTUNE_API_TOKEN')

modelLoadFlag = True

subsampleFreq = 64   # FINAL FREQUENCY IN HERTZ AFTER SUBSAMPLING
secondsInWindow = 1.
nperseg = subsampleFreq * secondsInWindow
noverlap = nperseg - 1
window = ('tukey', .25)

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# PARAMETERS - TRAINING

In [20]:
epochs = 5000
batch_size = 64
learningRate = 1e-5
loss_fn = nn.MSELoss()
optChoice = 'adam'

patience = 50
min_delta = 0

# UTILITY FUNCTIONS

In [4]:
# CONVERT STFT FROM R,THETA TO COMPLEX
# dim(z) = (# timesteps, # freq bins x 2 (2 reals = 1 complex))

def rThetaToComplex(z):
    rows, cols = z.shape
    shortTermFourier = np.zeros((rows, cols // 2), dtype=np.csingle)
    for i in range(rows):
        for k in range(cols // 2):
            r = z[i,k]
            theta = z[i, (k + cols // 2)]
            shortTermFourier[i,k] =  r * np.exp(complex(0, theta))
    return shortTermFourier.transpose() # dim = (# freq bins, # timepoints)

# CONVERT REAL STFT TO COMPLEX STFT, INVERT TO GET THE ISTFT (I.E. TIME SERIES), THEN PLOT

def realSTFTtoTimeSeries(realSTFT):
    shortTermFourierComplex = rThetaToComplex(realSTFT)
    times, inverseShortFourier = istft(shortTermFourierComplex, 
                                       fs=subsampleFreq, 
                                       window=window, 
                                       nperseg=nperseg, 
                                       noverlap=noverlap)
    return times, inverseShortFourier

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss

class EarlyStopper:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
class NeuralNetwork(nn.Module):
    
    def __init__(self, layerOrderedDict):
        super().__init__()
        self.model = nn.Sequential(layerOrderedDict)
        
    def forward(self, x):
        return self.model(x)

# LOAD NUMPY DATA ARRAYS

In [5]:
npzfile = np.load(arraySavePath)
x_trainRTheta = npzfile['x_trainRTheta']
x_validRTheta = npzfile['x_validRTheta'] 
y_trainRTheta = npzfile['y_trainRTheta'] 
y_validRTheta = npzfile['y_validRTheta']

_,nY = y_validRTheta.shape

trainXTensor = torch.Tensor(x_trainRTheta)
trainYTensor = torch.Tensor(y_trainRTheta)

trainDataset = TensorDataset(trainXTensor,trainYTensor)
trainDataLoader = DataLoader(trainDataset,batch_size=batch_size, shuffle=True)

validXTensor = torch.Tensor(x_validRTheta)
validYTensor = torch.Tensor(y_validRTheta)

validDataset = TensorDataset(validXTensor,validYTensor)
validDataLoader = DataLoader(validDataset,batch_size=batch_size, shuffle=True)


print("train: ")
for X, y in trainDataLoader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    
print("\ntest: ")
for X, y in validDataLoader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

train: 
Shape of X [N, C, H, W]: torch.Size([64, 5742])
Shape of y: torch.Size([64, 66]) torch.float32

test: 
Shape of X [N, C, H, W]: torch.Size([64, 5742])
Shape of y: torch.Size([64, 66]) torch.float32


# DEFINE OR LOAD THE MODEL

In [6]:
# Define model

if modelLoadFlag == True:
    model = torch.load(modelPath)
    bestTestLoss = test(validDataLoader, model, loss_fn)
else:
    layerOrderedDict = OrderedDict([('l1', nn.Linear(5742, 512)),
                                ('rl1', nn.ReLU()),
                                ('l2', nn.Linear(512, 512)),
                                ('lr2', nn.ReLU()),
                                ('l3', nn.Linear(512, 66))])
    
    model = NeuralNetwork(layerOrderedDict)
    bestTestLoss = float('inf')
    
    
model = model.to(device)
print(model)

Test Error: 
 Avg loss: 4.633556 

NeuralNetwork(
  (model): Sequential(
    (l1): Linear(in_features=5742, out_features=512, bias=True)
    (rl1): ReLU()
    (l2): Linear(in_features=512, out_features=512, bias=True)
    (lr2): ReLU()
    (l3): Linear(in_features=512, out_features=66, bias=True)
  )
)


# TRAIN (LOG DATA TO NEPTUNE)

In [21]:
if optChoice == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
elif optChoice == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
else:
    optimizer = None
    print('no optimizer chosen...')
    
early_stopper = EarlyStopper(patience=patience, min_delta=min_delta)

run = neptune.init_run(
    project=neptuneProject,
    api_token=api_token,  
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)

PARAMS = {
    "batch_size": batch_size,
    "learning_rate": learningRate,
    "optimizer": optChoice,
    "patience": patience,
    "min_delta": min_delta,
    "subsampleFreq": subsampleFreq,
    "secondsInWindow": secondsInWindow,
    "nperseg": nperseg,
    "noverlap": noverlap,
    "window": stringify_unsupported(window)
}
run["parameters"] = PARAMS

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(trainDataLoader, model, loss_fn, optimizer)
    test_loss = test(validDataLoader, model, loss_fn)
    
    if test_loss < bestTestLoss:
        bestTestLoss = test_loss
        torch.save(model, modelPath)
        run["model_best"].upload(modelPath)
        print("\nSaved a new best model!\n")
        
    run["train/loss"].append(train_loss)
    run["test/loss"].append(test_loss)
    
    if early_stopper.early_stop(test_loss):   
        print("Early stopping invoked....")
        break
        
run.stop()
print("Done!")

https://new-ui.neptune.ai/jettinger35/predictScalp/e/PRED-12
Epoch 1
-------------------------------
loss: 3.695208  [   64/89440]
Test Error: 
 Avg loss: 3.984244 

Epoch 2
-------------------------------
loss: 3.687844  [   64/89440]
Test Error: 
 Avg loss: 3.984945 

Epoch 3
-------------------------------
loss: 3.512255  [   64/89440]
Test Error: 
 Avg loss: 3.983126 

Epoch 4
-------------------------------
loss: 3.847361  [   64/89440]
Test Error: 
 Avg loss: 3.981781 

Epoch 5
-------------------------------
loss: 3.390279  [   64/89440]
Test Error: 
 Avg loss: 3.982100 

Epoch 6
-------------------------------
loss: 4.180506  [   64/89440]
Test Error: 
 Avg loss: 3.982082 

Epoch 7
-------------------------------
loss: 3.695006  [   64/89440]
Test Error: 
 Avg loss: 3.983579 

Epoch 8
-------------------------------
loss: 3.497055  [   64/89440]
Test Error: 
 Avg loss: 3.981409 

Epoch 9
-------------------------------
loss: 3.821937  [   64/89440]
Test Error: 
 Avg loss: 3.984

Test Error: 
 Avg loss: 3.939431 

Epoch 73
-------------------------------
loss: 4.001312  [   64/89440]
Test Error: 
 Avg loss: 3.942624 

Epoch 74
-------------------------------
loss: 3.663777  [   64/89440]
Test Error: 
 Avg loss: 3.941598 

Epoch 75
-------------------------------
loss: 3.503759  [   64/89440]
Test Error: 
 Avg loss: 3.940176 

Epoch 76
-------------------------------
loss: 3.847359  [   64/89440]
Test Error: 
 Avg loss: 3.940774 

Epoch 77
-------------------------------
loss: 3.587716  [   64/89440]
Test Error: 
 Avg loss: 3.940508 

Epoch 78
-------------------------------
loss: 3.889457  [   64/89440]
Test Error: 
 Avg loss: 3.941192 

Epoch 79
-------------------------------
loss: 3.710948  [   64/89440]
Test Error: 
 Avg loss: 3.946245 

Epoch 80
-------------------------------
loss: 4.214440  [   64/89440]
Test Error: 
 Avg loss: 3.944645 

Epoch 81
-------------------------------
loss: 4.083396  [   64/89440]
Test Error: 
 Avg loss: 3.944056 

Epoch 82
--

Test Error: 
 Avg loss: 3.927152 

Epoch 143
-------------------------------
loss: 3.179786  [   64/89440]
Test Error: 
 Avg loss: 3.926916 

Epoch 144
-------------------------------
loss: 3.551749  [   64/89440]
Test Error: 
 Avg loss: 3.928550 

Epoch 145
-------------------------------
loss: 3.316071  [   64/89440]
Test Error: 
 Avg loss: 3.926388 

Epoch 146
-------------------------------
loss: 3.785127  [   64/89440]
Test Error: 
 Avg loss: 3.925567 

Saved a new best model!
Epoch 147
-------------------------------
loss: 3.531979  [   64/89440]
Test Error: 
 Avg loss: 3.925649 

Epoch 148
-------------------------------
loss: 3.799834  [   64/89440]
Test Error: 
 Avg loss: 3.925434 

Saved a new best model!
Epoch 149
-------------------------------
loss: 3.578845  [   64/89440]
Test Error: 
 Avg loss: 3.923419 

Saved a new best model!
Epoch 150
-------------------------------
loss: 3.945816  [   64/89440]
Test Error: 
 Avg loss: 3.925492 

Epoch 151
---------------------------

Test Error: 
 Avg loss: 3.927823 

Epoch 219
-------------------------------
loss: 3.854582  [   64/89440]
Test Error: 
 Avg loss: 3.926114 

Epoch 220
-------------------------------
loss: 3.284026  [   64/89440]
Test Error: 
 Avg loss: 3.927172 

Epoch 221
-------------------------------
loss: 4.441832  [   64/89440]
Test Error: 
 Avg loss: 3.927553 

Epoch 222
-------------------------------
loss: 4.249127  [   64/89440]
Test Error: 
 Avg loss: 3.928012 

Epoch 223
-------------------------------
loss: 3.783826  [   64/89440]
Test Error: 
 Avg loss: 3.928942 

Epoch 224
-------------------------------
loss: 3.324861  [   64/89440]
Test Error: 
 Avg loss: 3.926971 

Epoch 225
-------------------------------
loss: 3.810966  [   64/89440]
Test Error: 
 Avg loss: 3.928077 

Epoch 226
-------------------------------
loss: 3.284823  [   64/89440]
Test Error: 
 Avg loss: 3.925314 

Epoch 227
-------------------------------
loss: 3.658512  [   64/89440]
Test Error: 
 Avg loss: 3.926401 

Ep

Explore the metadata in the Neptune app:
https://new-ui.neptune.ai/jettinger35/predictScalp/e/PRED-12/metadata
Done!


# Plot results of fit

In [ ]:
# PLOT PREDICTION VERSUS TRUTH

trainPlotFlag = False
    
if trainPlotFlag:
    x = trainXTensor
    y = y_trainRTheta
    trainTitle = 'train'
else:
    x = validXTensor
    y = y_validRTheta
    trainTitle = 'valididation'
    

x = validXTensor.to(device)
freqPredict = model(x).cpu().detach().numpy()

_, yPred = realSTFTtoTimeSeries(freqPredict)
_, yTrue = realSTFTtoTimeSeries(y)

lossTemp = loss_fn(torch.tensor(yPred), torch.tensor(yTrue)).item()
title = 'PyTorch: ' + trainTitle + ' (mse: %s)' % str(lossTemp)
plt.figure()
plt.plot(yPred, label='predict')
plt.plot(yTrue, label='true')
plt.legend()
plt.title(title)
plt.show()

# SCRATCH BELOW

In [ ]:
import neptune

# Create a Neptune run object
run = neptune.init_run(
    project='jettinger35/test',
    api_token=api_token,  
)

# Track metadata and hyperparameters by assigning them to the run
run["JIRA"] = "NPT-952"
run["algorithm"] = "ConvNet"

PARAMS = {
    "batch_size": 64,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
}
run["parameters"] = PARAMS

# Track the training process by logging your training metrics
for epoch in range(10):
    run["train/accuracy"].append(epoch * 0.6)  
    run["train/loss"].append(epoch * 0.4)

# Record the final results
run["f1_score"] = 0.66

# Stop the connection and synchronize the data with the Neptune servers
run.stop()
